In [97]:
import requests
import json
import pandas as pd

In [98]:
from requests_oauthlib import OAuth1

In [99]:
base_url = 'https://api.bricklink.com/api/store/v1'

In [100]:
xwing_set = '9493-1'
trading_post_set='6277-1'
medieval_market_village='10193-1'

In [101]:
auth = OAuth1('C589DC299F8C4B92BA09271DB9927312', 'FB1E18DD5A3A4DF0BA72A26E177ED3F4', 'B73EC794690D4C639CD34F55D6D32B32', '25D35B331A2F4FE0912789892C6233E1')

In [104]:
#oauth token for Clay Fitness
auth = OAuth1('C589DC299F8C4B92BA09271DB9927312', 'FB1E18DD5A3A4DF0BA72A26E177ED3F4', 'BAF7AB59684741F8B68E5FFFB8638324', '5A2F1618B68945FBAED02C1F11F43B9A')

In [105]:
set_response = requests.get(base_url + '/items/SET/' + medieval_market_village, auth=auth)

set_response_data = json.loads(set_response.text)['data']
set_response_data

{u'category_id': 9,
 u'dim_x': u'57.70',
 u'dim_y': u'37.20',
 u'dim_z': u'8.00',
 u'image_url': u'//img.bricklink.com/SL/10193-1.jpg',
 u'is_obsolete': False,
 u'name': u'Medieval Market Village',
 u'no': u'10193-1',
 u'thumbnail_url': u'//img.bricklink.com/S/10193-1.gif',
 u'type': u'SET',
 u'weight': u'1935.00',
 u'year_released': 2009}

In [106]:
response = requests.get(base_url + '/items/SET/{}/subsets'.format(trading_post_set), auth=auth)

In [107]:
response_data = json.loads(response.text)['data']

In [110]:
for j in response_data:
    entry = j['entries'][0]
    for i in ['N', 'U']:
        price_response_data = requests.get(base_url + '/items/{}/{}/price?color_id={}&new_or_used={}'.format(entry['item']['type'], entry['item']['no'], entry['color_id'], i), auth=auth)
        price_data = json.loads(price_response_data.text)['data']
        entry['avg_price_{}'.format(i)] = float(price_data['avg_price'])
        entry['qty_avg_price_{}'.format(i)] = float(price_data['qty_avg_price'])
        entry['min_price_{}'.format(i)] = float(price_data['min_price'])
        entry['max_price_{}'.format(i)] = float(price_data['max_price'])
        entry['total_quantity_{}'.format(i)] = price_data['total_quantity']
    print entry

{u'extra_quantity': 0, u'is_alternate': False, 'total_quantity_U': 11601, 'avg_price_U': 0.131, 'max_price_U': 1.95, 'max_price_': 1.95, 'avg_price_N': 0.3241, 'min_price_': 0.0259, 'max_price_N': 1.2521, 'min_price_U': 0.0259, 'total_quantity_': 11601, u'color_id': 3, u'item': {u'category_id': 18, u'type': u'PART', u'name': u'Minifig, Utensil Goblet', u'no': u'2343'}, u'is_counterpart': False, 'min_price_N': 0.1315, 'qty_avg_price_N': 0.2536, 'qty_avg_price_U': 0.129, 'avg_price_': 0.131, 'qty_avg_price_': 0.129, 'total_quantity_N': 3581, u'quantity': 2}
{u'extra_quantity': 0, u'is_alternate': False, 'total_quantity_U': 12077, 'avg_price_U': 0.0851, 'max_price_U': 1.25, 'max_price_': 1.25, 'avg_price_N': 0.1658, 'min_price_': 0.02, 'max_price_N': 0.891, 'min_price_U': 0.02, 'total_quantity_': 12077, u'color_id': 1, u'item': {u'category_id': 5, u'type': u'PART', u'name': u'Brick 2 x 2 Corner', u'no': u'2357'}, u'is_counterpart': False, 'min_price_N': 0.05, 'qty_avg_price_N': 0.1784, 'q

In [81]:
num_items = sum([x['entries'][0]['quantity'] for x in response_data])

## Price by sum of sub-items

In [92]:
avg_prices = [float(x['entries'][0]['quantity']) * float(x['entries'][0]['avg_price']) for x in response_data]
min_prices = [float(x['entries'][0]['quantity']) * float(x['entries'][0]['min_price']) for x in response_data ]
max_prices = [float(x['entries'][0]['quantity']) * float(x['entries'][0]['max_price']) for x in response_data]
min_prices_without_minifigs = [float(x['entries'][0]['quantity']) * float(x['entries'][0]['min_price']) for x in response_data if x['entries'][0]['item']['type'] != 'MINIFIG']
qty_avg_prices = [float(x['entries'][0]['quantity']) * float(x['entries'][0]['qty_avg_price']) for x in response_data]


KeyError: 'item'

In [82]:
num_items

587

In [93]:
sum(avg_prices)

250.3091

In [87]:
entries = [x['entries'][0] for x in response_data]
for e in entries:
    e['no'] = e['item']['no']
    e['name'] = e['item']['name']
    e['type'] = e['item']['type']
    e['category_id'] = e['item']['category_id']
    e.pop('item', None)

In [88]:
used_price_subset_df = pd.DataFrame(entries)

In [89]:
used_price_subset_df.sort_values(by='avg_price')

,avg_price,category_id,color_id,extra_quantity,is_alternate,is_counterpart,max_price,min_price,name,no,qty_avg_price,quantity,total_quantity,type
173,0.0315,27,5,0,False,False,0.3510,0.0054,"Plate, Modified 1 x 1 with Clip Vertical - Typ...",4085c,0.0292,4,57640,PART
8,0.0331,38,11,0,False,False,0.2341,0.0003,"Tile, Modified 1 x 2 with Handle",2432,0.0347,1,62213,PART
19,0.0332,27,11,0,False,False,1.0000,0.0003,"Plate, Modified 1 x 2 with Handle on Side - Fr...",2540,0.0316,2,142946,PART
174,0.0337,27,11,0,False,False,0.3600,0.0077,"Plate, Modified 1 x 1 with Clip Vertical - Typ...",4085c,0.0345,4,62837,PART
17,0.0351,27,5,0,False,False,0.8900,0.0084,"Plate, Modified 1 x 2 with Handle on Side - Fr...",2540,0.0352,1,30861,PART
113,0.0352,28,19,1,False,False,10.1153,0.0034,"Plate, Round 1 x 1 Straight Side",4073,0.0263,5,207968,PART
107,0.0354,28,11,0,False,False,1.1239,0.0045,"Plate, Round 2 x 2 with Axle Hole",4032,0.0355,3,104187,PART
67,0.0359,31,5,0,False,False,0.8000,0.0056,Slope 45 2 x 1,3040,0.0346,2,83849,PART
68,0.0364,8,3,0,False,False,0.7500,0.0036,"Brick, Round 1 x 1 Open Stud",3062b,0.0373,6,64615,PART
37,0.0367,5,3,0,False,False,1.4300,0.0078,Brick 1 x 1,3005,0.0387,18,148515,PART


In [73]:
sorted_df

,avg_price,category_id,color_id,extra_quantity,is_alternate,is_counterpart,max_price,min_price,name,no,qty_avg_price,quantity,total_quantity,type
181,0.0000,162,2,0,False,False,0.0000,0.0000,Cloth Sail 12 x 10 with Black Crossed Anchors ...,sailbb30,0.0000,1,0,PART
179,0.0000,162,2,0,False,False,0.0000,0.0000,"Cloth Sail 11 x 13, 5 Holes (Quadrilateral fro...",sailbb28,0.0000,1,0,PART
178,0.0000,162,1,0,False,False,0.0000,0.0000,"Cloth Sail 9 x 11, 3 Holes with Red Stripes Pa...",sailbb24,0.0000,1,0,PART
180,0.0000,162,2,0,False,False,0.0000,0.0000,Cloth Sail Triangular 11 x 15,sailbb29,0.0000,1,0,PART
170,0.0000,243,9,0,False,False,0.0000,0.0000,"Baseplate, Raised 32 x 32 with Ramp and Pit, W...",2552px3,0.0000,1,0,PART
113,0.0292,28,19,1,False,False,0.7000,0.0058,"Plate, Round 1 x 1 Straight Side",4073,0.0225,5,1681865,PART
112,0.0344,28,3,1,False,False,3.0000,0.0067,"Plate, Round 1 x 1 Straight Side",4073,0.0282,9,438009,PART
19,0.0377,27,11,0,False,False,1.0000,0.0084,"Plate, Modified 1 x 2 with Handle on Side - Fr...",2540,0.0359,2,181774,PART
8,0.0385,38,11,0,False,False,0.3934,0.0080,"Tile, Modified 1 x 2 with Handle",2432,0.0339,1,175516,PART
107,0.0415,28,11,0,False,False,0.3778,0.0065,"Plate, Round 2 x 2 with Axle Hole",4032,0.0398,3,217989,PART


In [103]:
set_response.text

u'{"meta":{"description":"TOKEN_IP_MISMATCHED: consumer: C589DC299F8C4B92BA09271DB9927312 IP: 69.193.185.110","message":"BAD_OAUTH_REQUEST","code":401}}'